# Prep_200

This notebook is a prep notebook for my research question 2 'A summary of  visualisations and packages use.' 

## Purpose
* In this notebook I will begin preparation on my research question 2 
* This research question is analysed with the 2nd larger dataset downloaded from - https://library.ucsd.edu/dc/object/bb2733859v as 'Notebook files - part 4' and 'Summary CSV data'

    * First I downloaded the data: CSV file containing notebook ID's and 200,000 individual ipynb files
    * Created functions that parsed the individual notebooks by their notebook ID in the CSV
    * Created a dataframe's with information I needed including: 
        * Packages used 
        * Headings
        * Display data
    * I saved the dataframe as a CSV file in /15487912_FYP/data/CSV_files/ for analysis in notebook Analysis_200

In [1]:
#importing relevant libraries
import os
import json
import numpy as np
import pandas as pd
import re

In [2]:
df_nb = pd.read_csv('../data/Dataset2/csv/notebooks.csv')

print('%s notebooks' % df_nb.shape[0])

1253620 notebooks


*The data above will not be found as was to large to upload*

* This is Adam Rule's large CSV file containing 1.2 million notebook ID's.
* As I am using only a snippet of this data I will cut this file to where I need it 
* The individual notebook files I downloaded had ID's from [400000:599997]

In [3]:
df_nb.index[df_nb['nb_id']==599997] #finding the index for notebook ID 599997

Int64Index([596266], dtype='int64')

In [4]:
df_nb_new = df_nb.iloc[397481:-657353] # cutting file to only have what I need 

In [5]:
df_nb_new.tail() 

,nb_id,html_url,max_filesize,min_filesize,name,path,query_page,repo_id
596262,599993,https://github.com/adithyap/mlcs16/blob/5ae2ba...,40630903,30145143,DataViewer-onlyF2D.ipynb,DataViewer-onlyF2D.ipynb,4,54208713
596263,599994,https://github.com/buds-lab/the-building-data-...,40630903,30145143,00_Temporal Data Exploration.ipynb,notebooks/00_Temporal Data Exploration.ipynb,4,58021880
596264,599995,https://github.com/ltauxe/GMT_Notebooks/blob/a...,40630903,30145143,GMT_chapter_13_examples.ipynb,GMT_chapter_13_examples.ipynb,4,80138033
596265,599996,https://github.com/dswirtz/UdacityCarND/blob/a...,40630903,30145143,P4_AdvancedLaneLines.ipynb,P4_AdvancedLaneFinding/P4_AdvancedLaneLines.ipynb,4,75776431
596266,599997,https://github.com/ofnanezn/MIR/blob/b1733290a...,40630903,30145143,Colombia Feature Vector-Copy1.ipynb,Colombia Feature Vector-Copy1.ipynb,4,87016623


Here we can see the dataframe goes to notebook 599997 and we can now use to iterate and parse the individual files

### Parsing notebooks
* Here I have created functions to parse the 200,000 notebooks.
    * I have extracted information about:
        * Packages used
        * Headings
        * Display data

### Function to find packages and headings

In [6]:
def get_cell_content_data(df):
    
    all_cells = []      

    for index, row in df.iterrows():

        f = '../data/Dataset2/notebooks/nb_%s.ipynb' % row['nb_id']
        with open(f) as nb_file:

            try:
                data = json.load(nb_file)
            except:
                continue

            if isinstance(data, dict): 
                keys = data.keys()
            else:
                keys = []
                
            # get the language
            nb_language = None
            if 'metadata' in keys:
                if isinstance(data, dict):
                    metadata_keys = data['metadata'].keys()
                else:
                    metadata_keys = []
            else:
                metadata_keys = []
            if 'language_info' in metadata_keys:
                if isinstance(data['metadata']['language_info'], dict):
                    lang_keys = data['metadata']['language_info'].keys()
                else:
                    lang_keys = None
                if 'name' in lang_keys:
                    nb_language = data['metadata']['language_info']['name']
            elif 'language' in metadata_keys:
                nb_language = data['metadata']['language']

            # get data for each cell, nbformat v 4.x
            if 'cells' in keys:
                for i, c in enumerate(data['cells']):
                    cell_data = get_cell_data(c, i, row['nb_id'], nb_language)
                    all_cells.append(cell_data)
            
            # get data for each cell, nbformat v 2.x / 3.x
            elif 'worksheets' in keys:
                for j, w in enumerate(data['worksheets']):
                    if isinstance(w, dict): 
                        worksheet_keys = w.keys()
                    else:
                        keys = []
                    if 'cells' in worksheet_keys:
                        for k, c in enumerate(w['cells']):
                            cell_data = get_cell_data(c, k, row['nb_id'], nb_language, j)
                            all_cells.append(cell_data)
                
    return all_cells

In [7]:
def get_cell_data(cell, index, nb_id, nb_language, worksheet_index = None):

    if isinstance(cell, dict): 
        cell_keys = cell.keys()
    else:
        cell_keys = [] 
    
    # get the cell type
    if 'cell_type' in cell_keys:
        cell_type = cell['cell_type']
    else:
        cell_type = None
    
    headings = []
    header_level = []
    imports = []
    
   # get the imports for python, Julia, and R
    if cell_type == 'code':
        
        lines_of_code = []
        
        if 'source' in cell_keys:
            if isinstance(cell['source'], list):
                lines_of_code = cell['source']
            elif isinstance(cell['source'], str):
                lines_of_code = cell['source'].splitlines()
            
        elif 'input' in cell_keys:
            if isinstance(cell['input'], list):
                lines_of_code = cell['input']
            elif isinstance(cell['input'], str):
                lines_of_code = cell['input'].splitlines()
                
        if nb_language == 'python':
            import_words = ['import', 'from']
        elif nb_language == 'R':
            import_words = ['library', 'require']
        elif nb_language in ['Julia', 'julia']:
            import_words = ['import', 'importall', 'using']
        else:
            import_words = []
            
        # get imports
        for l in lines_of_code:
            words = l.split()
            if len(words) >1:
                if words[0] in import_words:                    
                    imports.append(words[1])
    
    
    elif cell_type in ['heading', 'markdown']:
        
        
        # get the lines of markdown
        lines_of_markdown = []
        if 'source' in cell_keys:
            if isinstance(cell['source'], list):
                lines_of_markdown = cell['source']
            elif isinstance(cell['source'], str):
                lines_of_markdown = cell['source'].splitlines()
        
        # track headings in heading cells
        if 'level' in cell_keys:
            if 'source' in cell_keys:
                if len(lines_of_markdown) > 0:
                    headings.append([cell['level'], cell['source']])
        
        for l in lines_of_markdown:
            # get headings in markdown cells
            words = l.split()
            if len(words) > 2:
                if words[0] in ['#', '##', '###', '####', '#####', '######']:
                    headings.append(len(words[0]))
                    header_level.append(words[0])
                
    return [nb_id, nb_language, index, worksheet_index, len(imports), imports, len(headings), headings, header_level]

* The cell below takes 2 and a half hours to run
* I have also not included the large dataset in the data folder but can upload if needed.

In [8]:
# passing the notebook dataframe through function to iterate over each notebook and extract information
new_cell_data = get_cell_content_data(df_nb_new)

In [9]:
df_cells_new = pd.DataFrame(new_cell_data) # creating a dataframe with extracted information
print(df_cells_new.shape) # getting shape of dataframe

df_cells_new.columns = ['nb_id', 'nb_language', 'worksheet_index', 'cell_index', 'num_imports', 'imports', 
                    'num_headings', 'headings', 'header_level'] # renaming the columns

df_cells_new.head(5) # printing first 5 rows of dataframe

(4819241, 9)


,nb_id,nb_language,worksheet_index,cell_index,num_imports,imports,num_headings,headings,header_level
0,400000,julia,0,NaN,1,"[DataFrames,]",0,[],[]
1,400000,julia,1,NaN,0,[],0,[],[]
2,400000,julia,2,NaN,0,[],0,[],[]
3,400000,julia,3,NaN,0,[],0,[],[]
4,400000,julia,4,NaN,0,[],0,[],[]


### Function to find display data

In [11]:
def get_display_cell_data(df):
    
    all_cells = []      
   
    for index, row in df.iterrows():
     
        f = '../data/Dataset2/notebooks/nb_%s.ipynb' % row['nb_id']
        with open(f) as nb_file:
            
            try:
                data = json.load(nb_file)
            except:
                continue

            if isinstance(data, dict): 
                keys = data.keys()
            else:
                keys = []

            # get data for each cell, v 4.0
            if 'cells' in keys:
                for i, c in enumerate(data['cells']):
                    cell_data = get_display_data(c, i, row['nb_id'])
                    all_cells.append(cell_data)
            
            # get data for each cell, v 3.0
            elif 'worksheets' in keys:
                for j, w in enumerate(data['worksheets']):
                    
                    if isinstance(w, dict): 
                        worksheet_keys = w.keys()
                    else:
                        keys = []
                    
                    if 'cells' in worksheet_keys:
                        for k, c in enumerate(w['cells']):
                            cell_data = get_display_data(c, k, row['nb_id'], j)
                            all_cells.append(cell_data)
                
    return all_cells

In [12]:
def get_display_data(cell, index, nb_id, worksheet_index = None):
    
    if isinstance(cell, dict): 
        cell_keys = cell.keys()
    else:
        cell_keys = [] 
    
    if 'cell_type' in cell_keys:
        cell_type = cell['cell_type']
    else:
        cell_type = None
        
    # inilize output counts
    num_display_data = 0
    display_data_keys = []
    
    if 'outputs' in cell_keys:
        for o in cell['outputs']:

            if isinstance(o, dict):
                output_keys = o.keys()
            else:
                output_keys = []

            if 'output_type' in output_keys:
                if o['output_type'] == 'display_data':
                    num_display_data += 1
                    if 'source' in cell_keys:
                        if isinstance(cell['source'], list):
                            display_data_keys = cell['source']
                        elif isinstance(cell['source'], str):
                            display_data_keys = cell['source'].splitlines()
                        else:
                            display_data_keys = None
            
                    elif 'input' in cell_keys:
                        if isinstance(cell['input'], list):
                            display_data_keys = cell['input']
                        elif isinstance(cell['input'], str):
                            display_data_keys = cell['input'].splitlines()
                        else:
                            display_data_keys = None
        
    return [nb_id, worksheet_index, index, num_display_data, display_data_keys]

The cell below takes 2 and a half hours to run

In [13]:
# passing the notebook dataframe through function to iterate over each notebook and extract information
display_data = get_display_cell_data(df_nb_new)

In [14]:
df_display = pd.DataFrame(display_data) # creating a dataframe with extracted information
print(df_display.shape) # getting shape of dataframe

df_display.columns = ['nb_id', 'worksheet_index', 'cell_index', 'num_display_data', 'display_data_keys'] #renaming the columns

df_display.head(5) # printing first 5 rows of dataframe

(4819241, 5)


,nb_id,worksheet_index,cell_index,num_display_data,display_data_keys
0,400000,NaN,0,0,[]
1,400000,NaN,1,0,[]
2,400000,NaN,2,0,[]
3,400000,NaN,3,0,[]
4,400000,NaN,4,0,[]


### Saving this dataframe to a CSV file

In [10]:
df_cells_new.to_csv('../data/CSV_files/new_cell_info.csv', index=False, encoding='utf-8')

In [15]:
df_display.to_csv('../data/CSV_files/display_info.csv', index=False, encoding='utf-8')   

* Analysis of information in new_cell_info.csv will be looked at in notebook Analysis_201 in the analysis folder.
* Analysis of information in display_info.csv will be looked at in notebook Analysis_202 in the analysis folder.
* new_cell_info.csv created from this notebook will also be used for my 3rd and 4th research questions which will begin in Prep_300 and Prep_400 respectively.